# **Homework 3 - Places of the world**

- Ramona Tarantino 2082006
- Valeria Popolla 1918542
- Vahid Ghanbarizadehv



In [1]:
import lxml 
import os 
import requests 
import urllib.request 
import time 
import codecs 
import pandas as pd 
import csv 
from bs4 import BeautifulSoup 
from tqdm import tqdm 
from datetime import datetime


In [2]:
from google.colab import drive 

In [3]:
drive.mount('/content/drive/')

Mounted at /content/drive/


# **1**. Data collection
### **1.1.** Get the list of places

In [ ]:
link = open("/content/drive/MyDrive/Colab Notebooks/places.txt", "w")
for i in tqdm(range(1,401)):
  url = 'https://www.atlasobscura.com/places?page=' + str(i) + '&sort = likes_count'
  resp = requests.get(url)
  soup = BeautifulSoup(resp.content, features = 'lxml')
  for a in soup.find_all('a', class_= 'content-card content-card-place'):
    link.write(a.get('href')+'\n')
link.close()

In [6]:
#check the lines number 7200 OK! 
file1 = open("/content/drive/MyDrive/Colab Notebooks/places.txt", "r")

Lines = file1.readlines()
  
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    #print("Line{}: {}".format(count, line.strip()))
print(count)

7200


### **1.2.** Crawl places

In [ ]:
#Create a folder with 400 subfolders that contain .html files
for page in tqdm(range(1,401)):
  subfolder = "/content/drive/MyDrive/Colab Notebooks/html_page/pages_{}".format(page)
  os.makedirs(subfolder)

  f= open("/content/drive/MyDrive/Colab Notebooks/places.txt", 'r', encoding = "utf8")
  lines = f.readlines()[(page-1)*18:(page)*18]
  lines = [line.rstrip() for line in lines]
  f.close

  i=1+18*(page-1)
  #for each link we read we create a new .html file
  for link in lines: 
    link = 'https://www.atlasobscura.com'+ link
    html = requests.get(link)

    file_name = '{}/{}.html'.format(subfolder, i)
    g= open(file_name, 'w', encoding= "utf8")
    g.write(html.text)
    i+=1


In [4]:
#we initialize the empty lists
placeName=[]
placeTags=[]
listvisited =[]
listwant =[]
listdescription=[]
listurl =[]
listdate =[]
listshort_description =[]
listplaceAlt=[]
listaddr=[]
listplaceLong=[]
editors = []
placeNearby=[]
placeRelatedLists=[]
placeRelatedPlaces=[]

### **1.3** Parse downloaded pages



In [ ]:
for m in tqdm(range(1,401)):
  files = os.listdir('/content/drive/MyDrive/Colab Notebooks/html_page/page_'+ str(m))
  print(m)
  for k in range(18*(m-1)+1, (18*m+1)):
    print (k)
    file = open("/content/drive/MyDrive/Colab Notebooks/html_page/page_" + str(m) + '/' + str(k) + '.html', 'r', encoding = "utf8")
    soup = BeautifulSoup(file, 'lxml')

    #PLACE NAME 
    try:
      placeNames = soup.find('h1', {'class': 'DDPage__header-title'}).text
     
      placeName.append(placeNames)
    except: 
      placeName.append('')

    #PLACE TAGS 
    try:
      tag= soup.find_all('a', {'class': 'itemTags__link js-item-tags-link'})
      tag = [m.get('href').split('/categories/')[1] for m in tag]
      placeTags.append(tag)
      
    except: 
      placeTags.append('')

    #numPeopleVisited

    try:
      visited = int(soup.find_all('div', {'class':'title-md item-action-count'})[0].contents[0])
      listvisited.append(visited)
    except: 
      listvisited.append('')

    #numPeopleWant
    try:
      want = int(soup.find_all('div', {'class':'title-md item-action-count'})[1].contents[0])
      listwant.append(want)
    except: 
      listwant.append('')

    #placeDesc
    try:
      description = soup.find_all('div',{'class':'DDP__body-copy'})[0].getText().strip().replace('\n', ' ').replace('\xa0', ' ')
      listdescription.append(description)
    except: 
      listdescription.append('')
    #placeShortDesc


    try:
      short_description = soup.find_all('h3', {'class':'DDPage__header-dek'})[0].contents[0].replace('\n', ' ').replace('\xa0', ' ')
      listshort_description.append(short_description)
    except: 
      listshort_description.append('')

    #placeNearby
    try:
      pnb = soup.find_all('div', {'class': 'DDPageSiderailRecirc__item-title'})
      pnb = [i.get_text() for i in pnb]
      placeNearby.append(pnb)
    except: 
      placeNearby.append('')

    #placeAddress
    try:
      addr = soup.find_all('address', {'class': 'DDPageSiderail__address'})[0]
      addr= addr.find('div').get_text(separator=' ').split('\n')[0]
      listaddr.append(addr)
    except:
      listaddr.append('')

    #placeALT 
    try:
      alt_long = soup.find('div', {'class': 'DDPageSiderail__coordinates js-copy-coordinates'}).contents[2].replace('\n','')
      al = alt_long.split(',')
      placeAlt = float(al[0])
      listplaceAlt.append(placeAlt)
    except:
      listplaceAlt.append('')

    #placeLong
    try:
      p_long = soup.find('div', {'class': 'DDPageSiderail__coordinates js-copy-coordinates'}).contents[2].replace('\n','')
      al2 = p_long.split(',')
      placeLong = float(al2[1])
      listplaceLong.append(placeLong)
    except:
      listplaceLong.append('')

    #placePubDate
    try:
      date = soup.find('div', {'class': "DDPContributor__name"}).getText()
      date = pd.to_datetime(date)
      listdate.append(date)
    except:
      listdate.append('')


    #placeEditors
    try:
      ed = soup.find_all('div', {'class': 'ugc-editor-icons'})
      ed=[i.findChildren('a')[0].get('href') for i in ed]
      ed=[j.split('/users')[1] for j in ed]
      editors.append(ed)
    except: 
      editors.append('')

    #placeRelatedLists
    try:
      prl = soup.find_all('div', {'data-gtm-template': 'DDP Footer Recirc Lists'})[0]
      prl = [i.text.strip() for i in prl.find_all('h3')]
      placeRelatedLists.append(prl)
    except:
      placeRelatedLists.append('')

    #placeRelatedPlaces
    try:
      rep = soup.find_all('div', {'class':'CardRecircSection__title'})
      rep = [i for i in rep if i.text == 'Related Places'][0]
      rep = rep.find_all_next('div',{'class': 'Card__action-btns vue-js-been-there-everywhere-place'})
      rep = [j.get('data-place-title') for j in rep]
      placeRelatedPlaces.append(rep)
    except:
      placeRelatedPlaces.append('')

    #placeURL
    try:
      url = soup.find('meta', attrs = {'property':'og:url'})
      url = url['content']
      listurl.append(url)
    except:
      listurl.append('')

    




In [6]:
#TEST 
print(len(placeName))
print(len(placeTags))
print(len(editors))
print(len(listvisited))
print(len(listwant))
print(len(listdescription))
print(len(listurl ))
print(len(listdate ))
print(len(listshort_description ))
print(len(listplaceAlt))
print(len(listaddr))
print(len(listplaceLong))
print(len(placeNearby))
print(len(placeRelatedLists))
print(len(placeRelatedPlaces))



7200
7200
7200
7200
7200
7200
7200
7200
7200
7200
7200
7200
7200
7200
7200


In [7]:
colonne = ['placeName', 'placeTags', 'placeNearby', 'editors', 'placeRelatedLists', 'placeRelatedPlaces',
        'listvisited', 'listwant', 'listdescription', 'listurl', 'listdate', 'listshort_description',
        'listplaceAlt', 'listaddr', 'listplaceLong']
types = {'placeName': 'object', 
         'placeTags': 'object', 
         'placeNearby': 'object', 
         'editors': 'object', 
         'placeRelatedLists': 'object', 
         'placeRelatedPlaces': 'object',
        'listvisited': 'object', 
         'listwant': 'object', 
         'listdescription': 'object',
         'listurl': 'object', 
         'listdate':'datetime64',
         'listshort_description': 'object',
        'listplaceAlt': 'int64',
         'listaddr': 'object',
         'listplaceLong':'int64'}

In [8]:
dt = pd.DataFrame(columns= colonne).astype(dtype = types)
dt.placeName = placeName
dt.placeTags=placeTags
dt.placeNearby=placeNearby
dt.editors = editors
dt.placeRelatedLists= placeRelatedLists
dt.placeRelatedPlaces=placeRelatedPlaces
dt.listvisited =listvisited
dt.listwant =listwant
dt.listdescription=listdescription
dt.listurl =listurl
dt.listdate =listdate
dt.listshort_description =listshort_description
dt.listplaceAlt=listplaceAlt
dt.listaddr=listaddr
dt.listplaceLong=listplaceLong


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  if sys.path[0] == '':


In [13]:
dt = dt.to_csv("/content/drive/MyDrive/Colab Notebooks/places2.tsv", sep='\t', index=False)

In [15]:
dt = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/places2.tsv", sep='\t')
dt.head(5)

,placeName,placeTags,placeNearby,editors,placeRelatedLists,placeRelatedPlaces,listvisited,listwant,listdescription,listurl,listdate,listshort_description,listplaceAlt,listaddr,listplaceLong
0,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...","['African Burial Ground National Monument', 'T...","['/rebekah-otto', '/annetta-black']",['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",1828.0,8579.0,The first New York City subway was built and o...,http://www.atlasobscura.com/places/city-hall-s...,2010-05-08,A beautiful and abandoned New York subway stat...,40.7134,"31 Centre St New York, New York, 10007 United ...",-74.0046
1,Highgate Cemetery,"['rivals-week', 'vampires', 'hoaxes', 'occult'...","[""World's Largest Potted Plant"", 'Dick Whittin...",['/annetta-black'],"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",2617.0,8182.0,"Opened in 1839, Highgate is one of London’s mo...",http://www.atlasobscura.com/places/highgate-ce...,2014-08-09,London's creepiest cemetery was once the site ...,51.5675,"Swain's Lane, Highgate London, England, N6 Uni...",-0.1483
2,Leadenhall Market,"['markets', 'film-locations', 'film']","['The Cornhill Devils ', ""London's Original an...","['/meghanneal', '/643fc779-d0c4-40f3-9fc2-3637...","['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",3134.0,7567.0,The ornate 19th-century painted roof and cobbl...,http://www.atlasobscura.com/places/leadenhall-...,2016-08-01,This ornate Victorian marketplace was the sett...,51.5126,"London, England, EC3V United Kingdom",-0.0834
3,The Wave Organ,"['sea-organ', 'aletrail', 'sounds', 'music', '...","['Long Now Orrery', 'The Stern of the Briganti...","['/mbison', '/catleast']","[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",2427.0,7406.0,"Located on a jetty in the San Francisco Bay, t...",http://www.atlasobscura.com/places/wave-organ,2008-11-21,A huge musical instrument played by the ocean.,37.8085,"83 Marina Green Dr San Francisco, California, ...",-122.4401
4,Catacombes de Paris,"['ossuaries', 'memento-mori', 'catacombs-and-c...","['Sculptures de Décure', 'Arago Medallions', ""...","['/cpilgrim', '/escape-zeppelin']","['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",4444.0,7044.0,"In 2004, Parisian police were assigned to do a...",http://www.atlasobscura.com/places/catacombes-...,2009-02-13,"The vast, legendary catacombs hold secrets muc...",48.8343,"1 Place Denfert-Rochereau Paris, 75014 France",2.3322


# **2**. Search Engine


In [8]:
import nltk
import pandas as pd
from nltk.corpus import wordnet
from os.path import expanduser
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import string

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [9]:
# Stopwords and punctuation removal
def stopping(ls, *args):
    processed = []
    for case in ls:
        tok = word_tokenize(case)
        punctuation = str.maketrans('', '', string.punctuation)
        remove_punc = [w.translate(punctuation) for w in tok]
        stop_words = set(stopwords.words("english"))
        res = [x for x in remove_punc if not x in stop_words]
        result = ""
        for x in res:
            result = result + " " + x
        processed.append(result)
    return processed

Stemming and lemmatization are methods used by search engines and chatbots to analyze the meaning behind a word. Stemming uses the stem of the word, while lemmatization uses the context in which the word is being used. We'll later go into more detailed explanations and examples.  
- Lemmatization takes into consideration the morphological analysis of the words. To do so, it is necessary to have detailed dictionaries which the algorithm can look through to link the form back to its lemma.
- Stemming algorithms work by cutting off the end or the beginning of the word, taking into account a list of common prefixes and suffixes that can be found in an inflected word. This indiscriminate cutting can be successful in some occasions, but not always, and that is why we affirm that this approach presents some limitations. 

In [10]:
# Lemmatizer 
def lemma(desc):
    out=[]
    lemmatizer = WordNetLemmatizer()
    res = [lemmatizer.lemmatize(w) for w in desc]
    result = ""
    for x in res:
        result = result + " " + x
    out.append(result.lower())  
    return out

In [11]:
from nltk.stem import PorterStemmer 
ps = PorterStemmer()
def Stemming(desc):
    return [ps.stem(w) for w in desc]




In [12]:
descr_corpus_tokenized = []
tokenizer = RegexpTokenizer(r'\w+')

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/places2.tsv", sep='\t')

for descr in df.listdescription:
    tokenized_descr = tokenizer.tokenize(str(descr).lower())
    descr_corpus_tokenized.append(tokenized_descr)

for s,descr in enumerate(descr_corpus_tokenized):
    filtered = []
    for token in descr:
        if len(token) > 2 and not token.isnumeric():
            filtered.append(token)
    descr_corpus_tokenized[s] = filtered

In [13]:
pd.options.mode.chained_assignment = None
#Create a new column with the preprocessed text    
df["Placesdescription"] = pd.Series('object')

#Assign the new description to each new row as a list
for y in range(len(df.listdescription)):
    df.Placesdescription[y] = [x for x in stopping(lemma(descr_corpus_tokenized[y])) if x != ""]

#Convert lists into strings for each row     
for x in range(len(df.Placesdescription)):
    s = ""
    for y in df.Placesdescription[x]:
        s+=y
    df.Placesdescription[x] = s

In [14]:
df.head(5)
#dataframe with the new column "Placesdescription" 

,placeName,placeTags,placeNearby,editors,placeRelatedLists,placeRelatedPlaces,listvisited,listwant,listdescription,listurl,listdate,listshort_description,listplaceAlt,listaddr,listplaceLong,Placesdescription
0,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...","['African Burial Ground National Monument', 'T...","['/rebekah-otto', '/annetta-black']",['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",1828.0,8579.0,The first New York City subway was built and o...,http://www.atlasobscura.com/places/city-hall-s...,2010-05-08,A beautiful and abandoned New York subway stat...,40.7134,"31 Centre St New York, New York, 10007 United ...",-74.0046,first new york city subway wa built operated ...
1,Highgate Cemetery,"['rivals-week', 'vampires', 'hoaxes', 'occult'...","[""World's Largest Potted Plant"", 'Dick Whittin...",['/annetta-black'],"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",2617.0,8182.0,"Opened in 1839, Highgate is one of London’s mo...",http://www.atlasobscura.com/places/highgate-ce...,2014-08-09,London's creepiest cemetery was once the site ...,51.5675,"Swain's Lane, Highgate London, England, N6 Uni...",-0.1483,opened highgate one london infamous cemetery ...
2,Leadenhall Market,"['markets', 'film-locations', 'film']","['The Cornhill Devils ', ""London's Original an...","['/meghanneal', '/643fc779-d0c4-40f3-9fc2-3637...","['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",3134.0,7567.0,The ornate 19th-century painted roof and cobbl...,http://www.atlasobscura.com/places/leadenhall-...,2016-08-01,This ornate Victorian marketplace was the sett...,51.5126,"London, England, EC3V United Kingdom",-0.0834,ornate 19th century painted roof cobbled floo...
3,The Wave Organ,"['sea-organ', 'aletrail', 'sounds', 'music', '...","['Long Now Orrery', 'The Stern of the Briganti...","['/mbison', '/catleast']","[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",2427.0,7406.0,"Located on a jetty in the San Francisco Bay, t...",http://www.atlasobscura.com/places/wave-organ,2008-11-21,A huge musical instrument played by the ocean.,37.8085,"83 Marina Green Dr San Francisco, California, ...",-122.4401,located jetty san francisco bay wave organ wa...
4,Catacombes de Paris,"['ossuaries', 'memento-mori', 'catacombs-and-c...","['Sculptures de Décure', 'Arago Medallions', ""...","['/cpilgrim', '/escape-zeppelin']","['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",4444.0,7044.0,"In 2004, Parisian police were assigned to do a...",http://www.atlasobscura.com/places/catacombes-...,2009-02-13,"The vast, legendary catacombs hold secrets muc...",48.8343,"1 Place Denfert-Rochereau Paris, 75014 France",2.3322,parisian police assigned training exercise pr...


In [28]:
df['Placesdescription'].head()

0     first new york city subway wa built operated ...
1     opened highgate one london infamous cemetery ...
2     ornate 19th century painted roof cobbled floo...
3     located jetty san francisco bay wave organ wa...
4     parisian police assigned training exercise pr...
Name: Placesdescription, dtype: object

#**2.1** Conjunctive query

## **2.1.1)** Create your index!

In [ ]:
dictionary = {}
controllo = []
indice=1

for parola in df.Placesdescription:
  newparola = parola.split()
  for parola in newparola:
    if parola not in controllo:
      dictionary[parola]=indice
      controllo.append(parola)
      indice+=1

 

In [125]:
#create a new file pickle to save the vocabulary
import pickle 
with open("/content/drive/MyDrive/Colab Notebooks/vocabulary.pkl", 'wb' ) as v:
  pickle.dump(dictionary, v)

In [130]:
print(dictionary)

{'first': 1, 'new': 2, 'york': 3, 'city': 4, 'subway': 5, 'wa': 6, 'built': 7, 'operated': 8, 'interborough': 9, 'rapid': 10, 'transit': 11, 'company': 12, 'irt': 13, 'opened': 14, 'october': 15, 'joy': 16, 'elevated': 17, 'train': 18, 'streetcar': 19, 'rider': 20, 'hall': 21, 'station': 22, 'local': 23, 'track': 24, 'lavished': 25, 'fine': 26, 'architectural': 27, 'detail': 28, 'including': 29, 'glass': 30, 'tile': 31, 'large': 32, 'chandelier': 33, 'however': 34, 'gustavino': 35, 'vaulted': 36, 'ceiling': 37, 'skylight': 38, 'lost': 39, 'busy': 40, 'commuter': 41, 'stop': 42, 'one': 43, 'least': 44, 'used': 45, 'system': 46, 'turnstile': 47, 'installed': 48, 'nearby': 49, 'brooklyn': 50, 'bridge': 51, 'frequented': 52, 'express': 53, 'closer': 54, 'connecting': 55, 'curved': 56, 'platform': 57, 'car': 58, 'center': 59, 'door': 60, 'could': 61, 'unless': 62, 'specially': 63, 'modified': 64, 'control': 65, 'allowed': 66, 'end': 67, 'closed': 68, 'along': 69, 'line': 70, 'lengthened': 7

In [135]:
#create a new file cvs to save the vocabulary
import csv
with open('/content/drive/MyDrive/Colab Notebooks/vocabulary.csv', 'w') as f:  
    w = csv.writer(f)
    w.writerows(dictionary.items())

In [138]:
with open('/content/drive/MyDrive/Colab Notebooks/vocabulary.csv', 'w', newline = '',encoding = 'utf8') as f:
    fieldnames = ['Word', 'term_id']
    w = csv.DictWriter(f, fieldnames = fieldnames,)
    w.writeheader()
    for key in dictionary:
        w.writerow({"Word": key, 'term_id': dictionary[key]})
        

In [35]:
vocabulary = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/vocabulary.csv")
vocabulary.head()
#test print 

,Word,term_id
0,first,1
1,new,2
2,york,3
3,city,4
4,subway,5


### Inverted Index

In [13]:
with open("/content/drive/MyDrive/Colab Notebooks/vocabulary.pkl", "rb") as file: 
  voc = pickle.load(file)

In [23]:
from collections import defaultdict
n = defaultdict(list)

for key, value in tqdm(voc.items()):
  for i in range(7200):
    if key in str(df.Placesdescription[i]).split():
      n[value].append(i)


100%|██████████| 45653/45653 [2:02:17<00:00,  6.22it/s]


In [25]:
#save the inverted index in a file pickle
with open("/content/drive/MyDrive/Colab Notebooks/inverted_index.pkl", 'wb' ) as idx:
  pickle.dump(n, idx)

In [14]:
with open("/content/drive/MyDrive/Colab Notebooks/inverted_index.pkl", "rb") as files: 
  inverted_index = pickle.load(files)

In [15]:
#create a file csv to test
with open('/content/drive/MyDrive/Colab Notebooks/inverted_index.csv', 'w', newline = '',encoding = 'utf8') as f:
    fieldnames = ['term_id', 'document']
    w = csv.DictWriter(f, fieldnames = fieldnames,)
    w.writeheader()
    for key in inverted_index:
        w.writerow({"term_id": key, 'document': inverted_index[key]})

In [34]:
index = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/inverted_index.csv")
index.head()

,term_id,document
0,1,"[0, 1, 4, 19, 20, 21, 29, 30, 33, 37, 38, 44, ..."
1,2,"[0, 9, 10, 11, 17, 18, 19, 20, 21, 26, 28, 32,..."
2,3,"[0, 18, 19, 36, 37, 47, 50, 57, 61, 68, 69, 71..."
3,4,"[0, 1, 3, 4, 7, 8, 12, 14, 18, 23, 26, 28, 30,..."
4,5,"[0, 7, 69, 72, 97, 253, 298, 347, 371, 380, 47..."


In [15]:
#function to clean the query
stop_words = stopwords.words("english")
def clean_query(query):
    cleaned_query = query.lower()
    cleaned_query = word_tokenize(cleaned_query)
    cleaned_query = [w for w in cleaned_query if w.isalnum()]
    cleaned_query = [w for w in cleaned_query if w not in stop_words]
    cleaned_query = [ps.stem(w) for w in cleaned_query]
    
    return cleaned_query

In [150]:
def match(query): 
  with open("/content/drive/MyDrive/Colab Notebooks/vocabulary.pkl", 'rb') as f: 
    vocab = pickle.load(f)

    query_index=[]
    for word in clean_query(query): 
      if word in vocab.keys(): 
        query_index.append(vocab[word])
    return query_index

In [151]:
import numpy as np
import ast
def searchEngine(query):
    with open("/content/drive/MyDrive/Colab Notebooks/inverted_index.pkl", 'rb') as f1: 
      inverted_index = pickle.load(f1)

    query_clean = match(query)
    doc = [set(inverted_index[key]) for key in query_clean if key in inverted_index.keys()]
    res = set.intersection(*doc)

    return res


    

In [152]:
def documentFinder(query):
    place = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/places2.tsv", sep='\t', usecols = ["placeName","listdescription", "listurl"])
    col = ['placeName', "listdescription", "listurl"]
    dataframe = pd.DataFrame(columns = col)
    ind = searchEngine(query)
        
    dataframe.placeName = [place.placeName[i] for i in ind]
    dataframe.listdescription = [place.listdescription[i] for i in ind]
    dataframe.listurl = [place.listurl[i] for i in ind]
    return dataframe 

- Dataframe with the descriptions column which contains the clean query

In [154]:
query= "american museum"
documentFinder(query)

,placeName,listdescription,listurl
0,The Wolfsonian-FIU,The Wolfsonian-Florida International Universit...,http://www.atlasobscura.com/places/wolfsonian-...
1,Hugh Mercer Apothecary Shop,"Hugh Mercer was a Scot, a warrior, a friend of...",http://www.atlasobscura.com/places/hugh-mercer...
2,National Museum of Health and Medicine,"Once housed in downtown Washington, D.C., the ...",http://www.atlasobscura.com/places/national-mu...
3,Lunch Box Museum,Lunch boxes bring back a certain sense of nost...,http://www.atlasobscura.com/places/lunch-box-m...
4,"Harriet Beecher Stowe, Slavery to Freedom Museum",This early brick Georgian townhouse sits incon...,http://www.atlasobscura.com/places/harriet-bee...
...,...,...,...
213,McNutt Sculpture Garden,A hidden sculpture garden exists just off the ...,http://www.atlasobscura.com/places/mcnutt-scul...
214,American Computer Museum,"“To collect, preserve, interpret, and display ...",http://www.atlasobscura.com/places/american-co...
215,Evel Knievel Museum,The Evel Knievel Museum takes you through the ...,http://www.atlasobscura.com/places/evel-knieve...
216,Akin Free Library,"The Akin Free Library, a gracious stone struct...",http://www.atlasobscura.com/places/akin-free-l...


 ## **2.2**) Conjunctive query & Ranking score

In [16]:
import heapq
import pickle
from nltk.tokenize import RegexpTokenizer
from numpy import inner
from numpy.linalg import norm
import numpy as np

In [19]:
with open("/content/drive/MyDrive/Colab Notebooks/vocabulary.pkl", 'rb') as f: 
    voc = pickle.load(f)

In [20]:
newdesc = []
tokenizer = RegexpTokenizer(r'\w+')
for descr in df.Placesdescription:
    tokenized_descr = tokenizer.tokenize(str(descr).lower())
    newdesc.append(tokenized_descr)

### Term Frequency (TF)
The number of times a word appears in a document divded by the total number of words in the document. Every document has its own term frequency.

In [21]:
tf={}
for i in range(len(newdesc)):
    tf[i] = {}
    for w in newdesc[i]:
        if w not in tf[i]:
            tf[i][w]=0
        tf[i][w]+=1

### Inverse Data Frequency (IDF)
The log of the number of documents divided by the number of documents that contain the word w. Inverse data frequency determines the weight of rare words across all documents in the corpus.

In [22]:
idf={}
for l in newdesc:
    for w in l:
        if w not in idf:
            idf[w]=0
        idf[w]+=1
#print(idf)
for key in idf:
    idf[key] =  np.log10(len(df.placeName)/idf[key])  

**2.2.1) Inverted index**

Create a new inverted index tf-idf

In [26]:
tfidf_invIndex = {}
for j in tqdm(range(len(df.placeName))):
    for w in tf[j]:
        if w not in list(tfidf_invIndex.keys()):
            tfidf_invIndex[w] = []
        tfidf_invIndex[w].append((j, round(tf[j][w] / idf[w], 3)))

100%|██████████| 7200/7200 [11:05<00:00, 10.81it/s]


In [27]:
#save the new inverted index in a file pickle
with open('/content/drive/MyDrive/Colab Notebooks/tfidfinvindex.pkl', 'wb') as f:
    pickle.dump(tfidf_invIndex, f)

In [23]:
with open('/content/drive/MyDrive/Colab Notebooks/tfidfinvindex.pkl', 'rb') as f:
  tfidf_invIndex = pickle.load(f)

2.2.2) Execute the query


In [17]:
queryold = "american museum"
query = clean_query(queryold)

In [24]:
#TF of words in the query
tfQuery= {}
for word in query:
    if word not in list(tfQuery.keys()):
        tfQuery[word]=0
    tfQuery[word]+=1
    
#Compute TFIDF values of each word in the query    
tfIdfQuery = {}
for w in list(tfQuery.keys()):
    tfIdfQuery[w] =   tfQuery[w]/np.log10(len(df.placeName)/idf[w])



In [44]:
tfIdfQuery

{'american': 0.2533074751717065, 'museum': 0.22791361548164485}

In [25]:
doc = []
for i in range(len(df.placeName)):
    for w in query:
        if w in list(tf[i].keys()):
            if w==query[-1]:
                doc.append(i)
        else:
            break

In [26]:
for i in doc:
    for word in tf[i]:
        tfidfD = [x[1] for x in tfidf_invIndex[word] if x[0]==i][0]

In data analysis, **cosine similarity** is a measure of similarity between two sequences of numbers. For defining it, the sequences are viewed as vectors in an inner product space, and the cosine similarity is defined as the cosine of the angle between them, that is, the dot product of the vectors divided by the product of their lengths. It follows that the cosine similarity does not depend on the magnitudes of the vectors, but only on their angle. The cosine similarity always belongs to the interval  [-1,1]. 



$$ cosine\ similarity = \frac{tfidf{query} \cdot tfidf_{doc}}{||query|| \cdot ||doc||}$$

 heap data structure

In [27]:
heap, k = [], 10
heapq.heapify(heap)
for i in doc:
    vector = list(set([x for x in list(tfIdfQuery.keys())] + [x for x in list(tf[i].keys())]))
    vectorQuery = dict.fromkeys(vector, 0)


    for key in tfIdfQuery:
        vectorQuery[key] = tfIdfQuery[key]
    vectorDoc = dict.fromkeys(vector, 0)


    for word in tf[i]:
        vectorDoc[word] = [x[1] for x in tfidf_invIndex[word] if x[0] == i][0]


    cosine = np.inner(list(vectorQuery.values()), list(vectorDoc.values())) / (norm(list(vectorQuery.values())) * norm(list(vectorDoc.values())))  
    heapq.heappush(heap,(cosine,i))


    if len(heap)==k+1:
        heapq.heappop(heap)
    heap  = [x for x in heap if float(x[1])!=0.0]

In [48]:
heap

[(0.5696512775471765, 3410),
 (0.5710834361529977, 5682),
 (0.5712816486014679, 2665),
 (0.5889827001911881, 1894),
 (0.5770605166269326, 6216),
 (0.6797387603482454, 4460),
 (0.5962530077824442, 3085),
 (0.6643818307841304, 141),
 (0.668968055911923, 6209),
 (0.5983491193063849, 1781)]

In [28]:

place2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/places2.tsv", sep='\t', usecols = ["placeName","listdescription", "listurl", "listvisited", "listaddr", "listplaceAlt", "listplaceLong"])
col = ['placeName', "Places_Description", "URL", 'listvisited', 'listaddr','alt', 'long', 'cosine_similarity']
dataframe2 = pd.DataFrame(columns = col)
l = len(heap)
ind = [heap[x][1] for x in range(l)]
    
dataframe2.placeName = [place2.placeName[i] for i in ind]
dataframe2.Places_Description = [place2.listdescription[i] for i in ind]
dataframe2.URL = [place2.listurl[i] for i in ind]
dataframe2.listvisited = [place2.listvisited[i] for i in ind]
dataframe2.listaddr = [place2.listaddr[i] for i in ind]
dataframe2.alt = [place2.listplaceAlt[i] for i in ind]
dataframe2.long = [place2.listplaceLong[i] for i in ind]


dataframe2.cosine_similarity = [heap[i][0] for i in range(l)]

final_df = dataframe2.sort_values(by=["cosine_similarity"], ascending=False)
final_df.index = range(l)
final_df

,placeName,Places_Description,URL,listvisited,listaddr,alt,long,cosine_similarity
0,American Writers Museum,The American Writers Museum—tucked away on the...,http://www.atlasobscura.com/places/american-wr...,180.0,"180 N Michigan Ave Chicago, Illinois, 60601 Un...",41.8856,-87.6249,0.679739
1,Siriraj Medical Museum,The Siriraj Medical Museum abounds with medica...,http://www.atlasobscura.com/places/siriraj-med...,246.0,"Siriraj Sub-District, Bangkoknoi District Bang...",13.7588,100.4851,0.668968
2,Museum of the Weird,The dime or dime store museum is by all accoun...,http://www.atlasobscura.com/places/museum-weird,1013.0,"Museum of the Weird 412 E 6th St. Austin, Texa...",30.2670,-97.7387,0.664382
3,National Cryptologic Museum,Update as of October 2021: Currently closed fo...,http://www.atlasobscura.com/places/national-cr...,320.0,"Canine Road Fort Meade, Maryland, 20755 United...",39.1117,-76.7711,0.598349
4,Museum of Russian Culture,"When you think San Francisco, you probably don...",http://www.atlasobscura.com/places/museum-of-r...,106.0,"2450 Sutter St Third Floor San Francisco, Cali...",37.7878,-122.4382,0.596253
5,National World War II Museum,"Perhaps once thought too narrowly focused, thi...",http://www.atlasobscura.com/places/national-wo...,1379.0,"945 Magazine Street New Orleans, Louisiana, 70...",29.9430,-90.0704,0.588983
6,Sweet Home Cafe,Thomas Downing was the oyster king. In 19th-ce...,http://www.atlasobscura.com/places/sweet-home-...,191.0,National Museum of African American History an...,38.8911,-77.0326,0.577061
7,Milwaukee Art Museum,Like the Guggenheim in New York and the Oaklan...,http://www.atlasobscura.com/places/milwaukee-a...,1648.0,"700 N. Art Museum Dr. Milwaukee, Wisconsin, 53...",43.0385,-87.8975,0.571282
8,Whale Museum,"Nestled in the San Juan Islands, the Whale Mus...",http://www.atlasobscura.com/places/whale-museum,275.0,"62 First St. N. Friday Harbor, Washington, 982...",48.5358,-123.0172,0.571083
9,Geppi's Entertainment Museum,It’s a unique place that can create a sentimen...,http://www.atlasobscura.com/places/geppi-s-ent...,254.0,"301 W Camden St Baltimore, Maryland, 21201 Uni...",39.2836,-76.6197,0.569651


# 3. Define a new score!
Now it's your turn. Build a new metric to rank places based on the queries of their users.

In this scenario, a single user can give input more information than a single textual query, so you need to consider all this information and think of a creative and logical way to answer the user's requests

We decided to define a new score that builds and improves upon the previously used cosine similarity. we assign different weights to a series of parameters,namely 
1. The cosine similarity between the query and the documents (which we had it in last question)
2. The number of people visited that place
3. The number of people want to visit there 
4. The queries' words are in our titles

#### OUR FORMULA

We decided to assign a doubble score for the maching of the title and the tokens inside the query. Exactly, for each anime, the score of matching () is figured by the ration of number of tokens inside the query and the title divided by the total number of token inside the query. And we add the possibility to specify what kind of anime the user is looking for. If an anime supply the request the score rest the same, instead if the type is different the score will be multiplied by 0.7 . The score of Popularity (the new one)is obtained by the product of the reciprocal of Popularity of an anime, times the number of members that it has, evrithing relativized.

**NEW SCORE: The final score of a document in our approach will be the sum of all of these values. The the ones that having higher score, will have a higher priority.**

In [29]:
#select the columns which we need for our new score in search engine 
place3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/places2.tsv", sep='\t', usecols = ["placeName","listdescription", "listurl","listvisited","listwant"])
place3.head()

,placeName,listvisited,listwant,listdescription,listurl
0,City Hall Station,1828.0,8579.0,The first New York City subway was built and o...,http://www.atlasobscura.com/places/city-hall-s...
1,Highgate Cemetery,2617.0,8182.0,"Opened in 1839, Highgate is one of London’s mo...",http://www.atlasobscura.com/places/highgate-ce...
2,Leadenhall Market,3134.0,7567.0,The ornate 19th-century painted roof and cobbl...,http://www.atlasobscura.com/places/leadenhall-...
3,The Wave Organ,2427.0,7406.0,"Located on a jetty in the San Francisco Bay, t...",http://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,4444.0,7044.0,"In 2004, Parisian police were assigned to do a...",http://www.atlasobscura.com/places/catacombes-...


In [30]:
#defining a newscore for our database base on the quries 
import math
def newscore(cosine, i, query):
    extra = 0
    #for normalizing we calculate the highest value in people visited that place
    largestvisitor = max(place3.listvisited) 
    largestwanter =  max(place3.listwant.apply(lambda x: pd.to_numeric(x, errors='coerce')))
    
    #we normalize listvisited and listwant in range [0,1]
    place3["normvisit"] = np.array(place3.listvisited) / largestvisitor
    place3["normwant"] = np.array((place3.listwant.apply(lambda x: pd.to_numeric(x, errors='coerce')))) / largestwanter                       
    if math.isnan(place3.normvisit[i]):
        place3.normvisit[i] = 0
    if math.isnan(place3.normwant[i]):
        place3.normwant[i] = 0
    #Upper Bound to 1 for cases exceeding 1
    cap = lambda x: 1.0 if x>1 else x
    #if a word in query is in our placename 
    wordinquery = [x in place3.placeName[i].lower() for x in query]
    if any(wordinquery):
        extra = (2.5/10)*wordinquery.count(True)/len(query)
    # calculate the final score based on number of people want to visit or visited and cosine similarity 
    return cap(extra + (6/10) * (cosine) + (1/10) * (place3.normvisit[i] )+(0.5/10) * (place3.normwant[i]))

In [53]:
place3

,placeName,listvisited,listwant,listdescription,listurl,normvisit,normwant
0,City Hall Station,1828.0,8579.0,The first New York City subway was built and o...,http://www.atlasobscura.com/places/city-hall-s...,0.275883,1.000000
1,Highgate Cemetery,2617.0,8182.0,"Opened in 1839, Highgate is one of London’s mo...",http://www.atlasobscura.com/places/highgate-ce...,0.394959,0.953724
2,Leadenhall Market,3134.0,7567.0,The ornate 19th-century painted roof and cobbl...,http://www.atlasobscura.com/places/leadenhall-...,0.472985,0.882038
3,The Wave Organ,2427.0,7406.0,"Located on a jetty in the San Francisco Bay, t...",http://www.atlasobscura.com/places/wave-organ,0.366284,0.863271
4,Catacombes de Paris,4444.0,7044.0,"In 2004, Parisian police were assigned to do a...",http://www.atlasobscura.com/places/catacombes-...,0.670691,0.821075
...,...,...,...,...,...,...,...
7195,Monument to a Century of Flight,536.0,405.0,"In a village in North Carolina’s Outer Banks, ...",http://www.atlasobscura.com/places/monument-to...,0.080893,0.047208
7196,Octagon Tower,179.0,405.0,Ely Cathedral dates as far back as the seventh...,http://www.atlasobscura.com/places/octagon-tower,0.027015,0.047208
7197,Taman Festival,58.0,405.0,"Eerie and overrun with lush greenery, this aba...",http://www.atlasobscura.com/places/taman-festi...,0.008753,0.047208
7198,The Cairo,342.0,405.0,Washington D.C.’s residential neighborhoods ar...,http://www.atlasobscura.com/places/the-cairo,0.051615,0.047208


In [31]:
#final dataframe which have new score 
new_df = final_df.copy()

sim = []

for i in range(len(final_df.cosine_similarity)):
    j = np.where(place3.placeName == final_df.placeName[i])[0][0]
    sim.append(newscore(final_df.cosine_similarity[i],j, query))

new_df.cosine_similarity = sim
new_df = new_df.sort_values(by=["cosine_similarity"], ascending=False)
new_df.index = range(l)
new_df

,placeName,Places_Description,URL,listvisited,listaddr,alt,long,cosine_similarity
0,American Writers Museum,The American Writers Museum—tucked away on the...,http://www.atlasobscura.com/places/american-wr...,180.0,"180 N Michigan Ave Chicago, Illinois, 60601 Un...",41.8856,-87.6249,0.664039
1,Museum of the Weird,The dime or dime store museum is by all accoun...,http://www.atlasobscura.com/places/museum-weird,1013.0,"Museum of the Weird 412 E 6th St. Austin, Texa...",30.2670,-97.7387,0.556169
2,Siriraj Medical Museum,The Siriraj Medical Museum abounds with medica...,http://www.atlasobscura.com/places/siriraj-med...,246.0,"Siriraj Sub-District, Bangkoknoi District Bang...",13.7588,100.4851,0.532798
3,National World War II Museum,"Perhaps once thought too narrowly focused, thi...",http://www.atlasobscura.com/places/national-wo...,1379.0,"945 Magazine Street New Orleans, Louisiana, 70...",29.9430,-90.0704,0.505071
4,Milwaukee Art Museum,Like the Guggenheim in New York and the Oaklan...,http://www.atlasobscura.com/places/milwaukee-a...,1648.0,"700 N. Art Museum Dr. Milwaukee, Wisconsin, 53...",43.0385,-87.8975,0.497484
5,National Cryptologic Museum,Update as of October 2021: Currently closed fo...,http://www.atlasobscura.com/places/national-cr...,320.0,"Canine Road Fort Meade, Maryland, 20755 United...",39.1117,-76.7711,0.494912
6,Museum of Russian Culture,"When you think San Francisco, you probably don...",http://www.atlasobscura.com/places/museum-of-r...,106.0,"2450 Sutter St Third Floor San Francisco, Cali...",37.7878,-122.4382,0.488816
7,Geppi's Entertainment Museum,It’s a unique place that can create a sentimen...,http://www.atlasobscura.com/places/geppi-s-ent...,254.0,"301 W Camden St Baltimore, Maryland, 21201 Uni...",39.2836,-76.6197,0.474797
8,Whale Museum,"Nestled in the San Juan Islands, the Whale Mus...",http://www.atlasobscura.com/places/whale-museum,275.0,"62 First St. N. Friday Harbor, Washington, 982...",48.5358,-123.0172,0.474720
9,Sweet Home Cafe,Thomas Downing was the oyster king. In 19th-ce...,http://www.atlasobscura.com/places/sweet-home-...,191.0,National Museum of African American History an...,38.8911,-77.0326,0.351817


In [35]:
new_df.to_csv('/content/drive/MyDrive/Colab Notebooks/dfquestion3new.csv')

### 3.4 Comparing

In this section we will compare the results that we got from this search engine the previous search engine that we have been built in the section 2.2.

1- The search engine in part 3.3 is more powerful than the search engine in 2.2 due to these:

2- Search engine in 3.3 for the resulted place that have the same cosine similarity will with the help of the other information of the place can sort the documents with respect to their popularity and title. The ones that are better will be higher in the result.

3- Search engine in 3.3 even if can not find any documents which contains all the words given for the description will give us the result based on the name in title and popularity of the places.

4- If for a query in description we have less than K related docuements, the search engine in 3.3 can give the result based on the other parameters which we have in our database.

# 4. Visualizing the most relevant places

Using maps can help people understand how far one place is from another so they can plan their trips more adequately. Here we challenge you to show a map with the places found with the score defined in point 3. Ensure you can at least identify and visualize the name, city, country, address and the number of people who visited each place. 

In [4]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.geocoders.base import Point
import folium
from folium.plugins import MarkerCluster

In [36]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dfquestion3new.csv', usecols=['placeName', 'listvisited', 'listaddr', 'alt', 'long'])
data

,placeName,listvisited,listaddr,alt,long
0,American Writers Museum,180.0,"180 N Michigan Ave Chicago, Illinois, 60601 Un...",41.8856,-87.6249
1,Museum of the Weird,1013.0,"Museum of the Weird 412 E 6th St. Austin, Texa...",30.2670,-97.7387
2,Siriraj Medical Museum,246.0,"Siriraj Sub-District, Bangkoknoi District Bang...",13.7588,100.4851
3,National World War II Museum,1379.0,"945 Magazine Street New Orleans, Louisiana, 70...",29.9430,-90.0704
4,Milwaukee Art Museum,1648.0,"700 N. Art Museum Dr. Milwaukee, Wisconsin, 53...",43.0385,-87.8975
5,National Cryptologic Museum,320.0,"Canine Road Fort Meade, Maryland, 20755 United...",39.1117,-76.7711
6,Museum of Russian Culture,106.0,"2450 Sutter St Third Floor San Francisco, Cali...",37.7878,-122.4382
7,Geppi's Entertainment Museum,254.0,"301 W Camden St Baltimore, Maryland, 21201 Uni...",39.2836,-76.6197
8,Whale Museum,275.0,"62 First St. N. Friday Harbor, Washington, 982...",48.5358,-123.0172
9,Sweet Home Cafe,191.0,National Museum of African American History an...,38.8911,-77.0326


In [37]:
data1 = data.copy()

In [38]:
data1[['Lat', 'Lng']] = data1[['alt', 'long']]
len(data1)

10

In [39]:
geolocator = Nominatim(user_agent="geoapiExercises")

In [40]:
def city_country(row):
    coord = f"{row['Lat']}, {row['Lng']}"
    location = geolocator.reverse(coord)
    address = location.raw['address']
    city = address.get('city', '')
    #state = address.get('state', '')
    country = address.get('country', '')
    row['city'] = city
    #row['state'] = state
    row['country'] = country
    return row

In [41]:
df4 = data1.apply(city_country, axis=1)

In [42]:
df4.to_csv('/content/drive/MyDrive/Colab Notebooks/df4.csv')

In [43]:
df4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/df4.csv")
df4

,Unnamed: 0,placeName,listvisited,listaddr,alt,long,Lat,Lng,city,country
0,0,American Writers Museum,180.0,"180 N Michigan Ave Chicago, Illinois, 60601 Un...",41.8856,-87.6249,41.8856,-87.6249,Chicago,United States
1,1,Museum of the Weird,1013.0,"Museum of the Weird 412 E 6th St. Austin, Texa...",30.2670,-97.7387,30.2670,-97.7387,Austin,United States
2,2,Siriraj Medical Museum,246.0,"Siriraj Sub-District, Bangkoknoi District Bang...",13.7588,100.4851,13.7588,100.4851,กรุงเทพมหานคร,ประเทศไทย
3,3,National World War II Museum,1379.0,"945 Magazine Street New Orleans, Louisiana, 70...",29.9430,-90.0704,29.9430,-90.0704,New Orleans,United States
4,4,Milwaukee Art Museum,1648.0,"700 N. Art Museum Dr. Milwaukee, Wisconsin, 53...",43.0385,-87.8975,43.0385,-87.8975,Milwaukee,United States
5,5,National Cryptologic Museum,320.0,"Canine Road Fort Meade, Maryland, 20755 United...",39.1117,-76.7711,39.1117,-76.7711,NaN,United States
6,6,Museum of Russian Culture,106.0,"2450 Sutter St Third Floor San Francisco, Cali...",37.7878,-122.4382,37.7878,-122.4382,San Francisco,United States
7,7,Geppi's Entertainment Museum,254.0,"301 W Camden St Baltimore, Maryland, 21201 Uni...",39.2836,-76.6197,39.2836,-76.6197,Baltimore,United States
8,8,Whale Museum,275.0,"62 First St. N. Friday Harbor, Washington, 982...",48.5358,-123.0172,48.5358,-123.0172,NaN,United States
9,9,Sweet Home Cafe,191.0,National Museum of African American History an...,38.8911,-77.0326,38.8911,-77.0326,Washington,United States


In [45]:
#empty map
world_map= folium.Map(Tiles='CartoDB positron',min_zoom=1.5)
marker_cluster = MarkerCluster().add_to(world_map)
#create a column with both latitude and longitude
df4['coord'] = list(zip(df4['Lat'], df4['Lng']))
#map each place with a point (if clicked infos will come up)
for i in range(len(df4)):
  location =  df4['coord'].values.tolist()[i]
  popup_text = """Name of the place : {}<br>
                    City : {}<br>
                    Country : {}<br>
                    People who visited : {}<br>
                    Address : {}<br>"""
  popup_text = popup_text.format(df4['placeName'][i],
                                   df4['city'][i],
                                   df4['country'][i],
                                   df4['listvisited'][i],
                                   df4['listaddr'][i]
                                   )
  folium.Marker(location, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map

In [46]:
map = world_map.save('/content/drive/MyDrive/Colab Notebooks/map.html')